In [1]:
## Built-in modules
import os

## Third party modules
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from collections import OrderedDict
%matplotlib inline

## Local modules
#pip install biogeme
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme import models
import biogeme.messaging as msg
import biogeme.tools as tools
import biogeme.results as res
from biogeme.expressions import Beta, DefineVariable, bioDraws, log, MonteCarlo

## Read in CFS2017 and Process the Dataset

In [3]:
df_raw = pd.read_csv('cfs_2017.csv')

In [5]:
%run CFS_National_dataprep_clean.ipynb # In this dataprep file, all modes are retained

In [6]:
df['mode_agg5'].value_counts()

For-hire Truck    2354704
Private Truck     1825760
Parcel            1565829
Air                 91854
Rail/IMX            62322
Other               60784
Name: mode_agg5, dtype: int64

In [7]:
df.shape

(5961253, 42)

In [8]:
# Only keep export shipments

df = df[df['EXPORT_YN'] == 'Y']

In [9]:
df['mode_decode'].value_counts(normalize=True)

Parcel             0.266308
For-hire Truck     0.251753
Air                0.228872
Truck and Water    0.195689
Other              0.014749
Rail/IMX           0.014730
Rail and Water     0.011859
Private Truck      0.011339
Deep Sea           0.004688
Great Lake         0.000014
Name: mode_decode, dtype: float64

In [10]:
# Generate mode categories tailored to exports

df['mode_exports'] = df['mode_agg5']
df.loc[df['mode_decode'] == 'Truck and Water', 'mode_exports'] = 'For-hire Truck'
df.loc[df['mode_decode'] == 'Rail and Water', 'mode_exports'] = 'Rail/IMX'

df['mode_exports'].value_counts(normalize=True)

C:\Users\hcyang\anaconda3\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
C:\Users\hcyang\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


For-hire Truck    0.447441
Parcel            0.266308
Air               0.228872
Rail/IMX          0.026589
Other             0.019451
Private Truck     0.011339
Name: mode_exports, dtype: float64

In [11]:
# Only consider "For-hire Truck", "Air", "Parcel", "Rail/IMX" in the mode choice estimation for exports

df = df[(df['mode_exports'] == 'For-hire Truck') | (df['mode_exports'] == 'Air') |
        (df['mode_exports'] == 'Parcel') | (df['mode_exports'] == 'Rail/IMX')]

In [12]:
sctg_dict = {1:'Animals and Fish ',
                2:'Cereal Grains ',
                3:'Agricultural Products ',
                4:'Animal Feed',
                5:'Meat',
                6:'Milled Grain Products',
                7:'Other Prepared Foodstuffs',
                8:'Alcoholic Beverages',
                9:'Tobacco Products',
                10:'Monumental or Building Stone',
                11:'Natural Sands',
                12:'Gravel and Crushed Stone ',
                13:'Other Non-Metallic Minerals',
                14:'Metallic Ores and Concentrates',
                15:'Coal',
                16:'Crude Petroleum',
                17:'Gasoline',
                18:'Fuel Oils ',
                19:'Other Coal and Petroleum Products',
                20:'Basic Chemicals',
                21:'Pharmaceutical Products',
                22:'Fertilizers',
                23:'Other Chemical Products and Preparations',
                24:'Plastics and Rubber',
                25:'Logs and Other Wood in the Rough',
                26:'Wood Products',
                27:'Pulp',
                28:'Paper or Paperboard Articles',
                29:'Printed Products',
                30:'Textiles',
                31:'Non-Metallic Mineral Products',
                32:'Base Metal',
                33:'Articles of Base Metal',
                34:'Machinery',
                35:'Electronic',
                36:'Motorized and Other Vehicles ',
                37:'Transportation Equipment',
                38:'Precision Instruments and Apparatus',
                39:'Furniture',
                40:'Misc. Manufactured Products',
                41:'Waste and Scrap ',
                43:'Mixed Freight'}

df['sctg_name'] = (df.SCTG).replace(sctg_dict).copy()

## Biogeme Estimation Setup 

In [13]:
# To run the mode choice model, drop SCTG == 37 'Transportation Equipment', becuase the mode is constrained by the type of equipment

df.drop(df[df['SCTG'] == 37].index, inplace = True)

### Create weight bin binary variables

In [14]:
df['wght_bin_1'] = (df['SHIPMT_WGHT'] <= 150).astype(int)
df['wght_bin_2'] = ((df['SHIPMT_WGHT'] > 150) & (df['SHIPMT_WGHT'] <= 1500)).astype(int)
df['wght_bin_3'] = ((df['SHIPMT_WGHT'] > 1500) & (df['SHIPMT_WGHT'] <= 30000)).astype(int)
df['wght_bin_4'] = ((df['SHIPMT_WGHT'] > 30000) & (df['SHIPMT_WGHT'] <= 45000)).astype(int)
df['wght_bin_5'] = (df['SHIPMT_WGHT'] > 45000).astype(int)

In [15]:
df.drop(df[(df['mode_exports'] == 'Parcel') & (df['wght_bin1'] == 2)].index, inplace = True)

In [16]:
df.groupby(['mode_exports'])['EXPORT_CNTRY'].value_counts().sort_index(ascending=True)

mode_exports    EXPORT_CNTRY
Air             A               19247
                C                3856
                E               15324
                M                1516
                S                4674
For-hire Truck  A               22635
                C               36834
                E               11199
                M               16114
                S                6888
Parcel          A               14553
                C               17514
                E               12733
                M                3234
                S                3751
Rail/IMX        A                1707
                C                1756
                E                 536
                M                1299
                S                 233
Name: EXPORT_CNTRY, dtype: int64

In [17]:
# Create export country dummies

df['EXPORT_CNTRY_A'] = (df['EXPORT_CNTRY'] == 'A').astype(int)
df['EXPORT_CNTRY_C'] = (df['EXPORT_CNTRY'] == 'C').astype(int)
df['EXPORT_CNTRY_E'] = (df['EXPORT_CNTRY'] == 'E').astype(int)
df['EXPORT_CNTRY_M'] = (df['EXPORT_CNTRY'] == 'M').astype(int)
df['EXPORT_CNTRY_S'] = (df['EXPORT_CNTRY'] == 'S').astype(int)

### Create the 'choice' and 'availability' variables

In [18]:
## alt_1 = Air, alt_2 = For-hire Truck, alt_3 = Parcel, alt_4 = Rail/IMX 

choice_dictionary ={'Air' : 1, 'For-hire Truck' : 2, 'Parcel' : 3, 'Rail/IMX': 4}
df['choice'] = df['mode_exports'].map(choice_dictionary).astype(int)

## NEW mode constraints on 02/07/2023 --> add the parcel filter back, but make air threshold at the max national processed sample
df['AV_1c'] = np.where(((df['SHIPMT_WGHT'] <= 15000) | (df['mode_exports'] == 'Air')), 1, 0) # the treshold is the air shipment clenaing criteria (~national max of cleaned dataset)
df['AV_2c'] = 1
df['AV_3c'] = np.where(((df['SHIPMT_WGHT'] <= 150) | (df['mode_exports'] == 'Parcel')), 1, 0)
df['AV_4c'] = 1


### Create Biogeme dataset (non-SHAP modeling)

In [19]:
df_short = df[['SHIPMT_ID','SHIPMT_DIST','SHIPMT_DIST_GC','SHIPMT_DIST_ROUTED','SHIPMT_WGHT_TON','SHIPMT_WGHT','value_density',
                'bulk','fuel_fert','interm_food','mfr_goods','other',
                'wholesale','mfring','mining','retail','info','management','transwarehouse',
                'choice','AV_1c','AV_2c','AV_3c','AV_4c',
                'WGT_FACTOR','wght_bin1',
                'wght_bin_1','wght_bin_2','wght_bin_3','wght_bin_4','wght_bin_5',
                'EXPORT_CNTRY_A','EXPORT_CNTRY_C','EXPORT_CNTRY_E','EXPORT_CNTRY_M','EXPORT_CNTRY_S']]

In [20]:
df_short.columns[df_short.isna().any()].tolist()

df_short = df_short.fillna(0).copy() # Biogeme does not allow NaN in dataset

In [21]:
# Check mode split using the subsample, to make sure each mode has decent number of obs retained. 
# alt_1 = Air, alt_2 = For-hire Truck, alt_3 = Parcel, alt_4 = Rail/IMX 

df_short['choice'].value_counts(normalize=True)

2    0.478878
3    0.264745
1    0.228100
4    0.028277
Name: choice, dtype: float64

In [22]:
database = db.Database('2017cfs', df_short)  

## The following statement allows you to use the names of the variable as Python variable.
globals().update(database.variables)

In [23]:
database.fullData

,SHIPMT_ID,SHIPMT_DIST,SHIPMT_DIST_GC,SHIPMT_DIST_ROUTED,SHIPMT_WGHT_TON,SHIPMT_WGHT,value_density,bulk,fuel_fert,interm_food,...,wght_bin_1,wght_bin_2,wght_bin_3,wght_bin_4,wght_bin_5,EXPORT_CNTRY_A,EXPORT_CNTRY_C,EXPORT_CNTRY_E,EXPORT_CNTRY_M,EXPORT_CNTRY_S
64,65,55,55,368,0.0295,59,249.118644,0,0,0,...,1,0,0,0,0,1,0,0,0,0
69,70,105,62,105,0.0025,5,150.600000,0,0,0,...,1,0,0,0,0,0,0,1,0,0
83,84,2384,2384,3069,0.0005,1,603.000000,0,0,0,...,1,0,0,0,0,0,1,0,0,0
112,113,96,96,349,0.6185,1237,7.554568,0,0,0,...,0,1,0,0,0,0,0,1,0,0
144,145,1582,1582,2375,0.0065,13,29.615385,0,0,0,...,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5978427,5978428,826,826,1318,0.0945,189,27.862434,0,0,0,...,0,1,0,0,0,0,0,1,0,0
5978439,5978440,95,58,95,0.0005,1,21.000000,0,0,0,...,1,0,0,0,0,0,0,1,0,0
5978483,5978484,174,141,174,21.3750,42750,1.629988,0,0,1,...,0,0,0,1,0,1,0,0,0,0
5978505,5978506,206,206,999,0.3575,715,15.437762,0,0,0,...,0,1,0,0,0,0,1,0,0,0


## Model Specifications

### Baseline MNL Model Specifications
#### note: There is no industry codes from FAF, re-estimate the mode choice after dropping the industry-related variables

In [24]:
# Parameters to be estimated
# (0, None, None, 0/1) --> (starting value, lower bound, upper bound, included/excluded in the estimation)
# Parameters to be estimated

ASC_AIR = Beta('ASC_AIR', 0, None, None, 0)
ASC_FHTRUCK = Beta('ASC_FHTRUCK', 0, None, None, 1)
ASC_PARCEL = Beta('ASC_PARCEL', 0, None, None, 0)
ASC_RAIL = Beta('ASC_RAIL', 0, None, None, 0)

B_AIR_WGHT_2 = Beta('B_AIR_WGHT_2', 0, None, None, 0)
B_RAIL_WGHT_2 = Beta('B_RAIL_WGHT_2', 0, None, None, 0)

B_AIR_WGHT_3 = Beta('B_AIR_WGHT_3', 0, None, None, 0)
B_RAIL_WGHT_3 = Beta('B_RAIL_WGHT_3', 0, None, None, 0)

B_RAIL_WGHT_4 = Beta('B_RAIL_WGHT_4', 0, None, None, 0)

B_RAIL_WGHT_5 = Beta('B_RAIL_WGHT_5', 0, None, None, 0)

B_AIR_VALDEN = Beta('B_AIR_VALDEN', 0, None, None, 0)
B_PARCEL_VALDEN = Beta('B_PARCEL_VALDEN', 0, None, None, 0)
B_RAIL_VALDEN = Beta('B_RAIL_VALDEN', 0, None, None, 0)

B_AIR_DIST = Beta('B_AIR_DIST', 0, None, None, 0)
B_PARCEL_DIST = Beta('B_PARCEL_DIST', 0, None, None, 0)
B_RAIL_DIST = Beta('B_RAIL_DIST', 0, None, None, 0)

B_AIR_CNTRY_A = Beta('B_AIR_CNTRY_A', 0, None, None, 0) 
B_PARCEL_CNTRY_A = Beta('B_PARCEL_CNTRY_A', 0, None, None, 0)  
B_RAIL_CNTRY_A = Beta('B_RAIL_CNTRY_A', 0, None, None, 0)

B_AIR_CNTRY_C = Beta('B_AIR_CNTRY_C', 0, None, None, 0) 
B_PARCEL_CNTRY_C = Beta('B_PARCEL_CNTRY_C', 0, None, None, 0)  
B_RAIL_CNTRY_C = Beta('B_RAIL_CNTRY_C', 0, None, None, 0)

B_AIR_CNTRY_M = Beta('B_AIR_CNTRY_M', 0, None, None, 0) 
B_PARCEL_CNTRY_M = Beta('B_PARCEL_CNTRY_M', 0, None, None, 0)  
B_RAIL_CNTRY_M = Beta('B_RAIL_CNTRY_M', 0, None, None, 0)

B_AIR_CNTRY_S = Beta('B_AIR_CNTRY_S', 0, None, None, 0) 
B_PARCEL_CNTRY_S = Beta('B_PARCEL_CNTRY_S', 0, None, None, 0)  
B_RAIL_CNTRY_S = Beta('B_RAIL_CNTRY_S', 0, None, None, 0)

B_AIR_BK = Beta('B_AIR_BK', 0, None, None, 0) 
B_PARCEL_BK = Beta('B_PARCEL_BK', 0, None, None, 0)  
B_RAIL_BK = Beta('B_RAIL_BK', 0, None, None, 0)

B_AIR_FF = Beta('B_AIR_FF', 0, None, None, 0)
B_PARCEL_FF = Beta('B_PARCEL_FF', 0, None, None, 0)
B_RAIL_FF = Beta('B_RAIL_FF', 0, None, None, 0) 

B_AIR_IF = Beta('B_AIR_IF', 0, None, None, 0)
B_PARCEL_IF = Beta('B_PARCEL_IF', 0, None, None, 0)
B_RAIL_IF = Beta('B_RAIL_IF', 0, None, None, 0) 

B_AIR_MG = Beta('B_AIR_MG', 0, None, None, 0)
B_PARCEL_MG = Beta('B_PARCEL_MG', 0, None, None, 0)
B_RAIL_MG = Beta('B_RAIL_MG', 0, None, None, 0)

B_AIR_INFO = Beta('B_AIR_INFO', 0, None, None, 0)
B_PARCEL_INFO = Beta('B_PARCEL_INFO', 0, None, None, 0)
B_RAIL_INFO = Beta('B_RAIL_INFO', 0, None, None, 0)

B_AIR_MFR = Beta('B_AIR_MFR', 0, None, None, 0)
B_PARCEL_MFR = Beta('B_PARCEL_MFR', 0, None, None, 0)
B_RAIL_MFR = Beta('B_RAIL_MFR', 0, None, None, 0)

B_AIR_MGT = Beta('B_AIR_MGT', 0, None, None, 0)
B_PARCEL_MGT = Beta('B_PARCEL_MGT', 0, None, None, 0)
B_RAIL_MGT = Beta('B_RAIL_MGT', 0, None, None, 0)

B_AIR_RETAIL = Beta('B_AIR_RETAIL', 0, None, None, 0)
B_PARCEL_RETAIL = Beta('B_PARCEL_RETAIL', 0, None, None, 0)
B_RAIL_RETAIL = Beta('B_RAIL_RETAIL', 0, None, None, 0)

B_AIR_TW = Beta('B_AIR_TW', 0, None, None, 0)
B_PARCEL_TW = Beta('B_PARCEL_TW', 0, None, None, 0)
B_RAIL_TW = Beta('B_RAIL_TW', 0, None, None, 0)

B_AIR_WS = Beta('B_AIR_WS', 0, None, None, 0)
B_PARCEL_WS = Beta('B_PARCEL_WS', 0, None, None, 0)
B_RAIL_WS = Beta('B_RAIL_WS', 0, None, None, 0)


In [25]:
# Definition of the utility functions
# Add export country dummies; replace SHIPMT_DIST with SHIPMT_DIST_ROUTED

V1 = ASC_AIR + \
     B_AIR_WGHT_2 * wght_bin_2 + B_AIR_WGHT_3 * wght_bin_3 + \
     B_AIR_DIST * SHIPMT_DIST_ROUTED + B_AIR_VALDEN * value_density + \
     B_AIR_CNTRY_C * EXPORT_CNTRY_C + B_AIR_CNTRY_M * EXPORT_CNTRY_M + B_AIR_CNTRY_S * EXPORT_CNTRY_S + \
     B_AIR_MG * mfr_goods  
    

V2 = ASC_FHTRUCK 

V3 = ASC_PARCEL + \
     B_PARCEL_DIST * SHIPMT_DIST_ROUTED + B_PARCEL_VALDEN * value_density + \
     B_PARCEL_CNTRY_A * EXPORT_CNTRY_A + B_PARCEL_CNTRY_C * EXPORT_CNTRY_C + B_PARCEL_CNTRY_M * EXPORT_CNTRY_M + B_PARCEL_CNTRY_S * EXPORT_CNTRY_S + \
     B_PARCEL_BK * bulk + B_PARCEL_FF * fuel_fert + B_PARCEL_IF * interm_food 
     

V4 = ASC_RAIL + \
     B_RAIL_WGHT_3 * wght_bin_3 + B_RAIL_WGHT_4 * wght_bin_4 + B_RAIL_WGHT_5 * wght_bin_5 + \
     B_RAIL_DIST * SHIPMT_DIST_ROUTED + B_RAIL_VALDEN * value_density + \
     B_RAIL_CNTRY_A * EXPORT_CNTRY_A + B_RAIL_CNTRY_M * EXPORT_CNTRY_M + B_RAIL_CNTRY_S * EXPORT_CNTRY_S + \
     B_RAIL_FF * fuel_fert + B_RAIL_IF * interm_food 

# Associate utility functions with the numbering of alternatives
V = {1: V1, 2: V2, 3: V3, 4: V4}

# Associate the availability conditions with the alternatives
av = {1: AV_1c, 2: AV_2c, 3: AV_3c, 4: AV_4c}

In [36]:
# Definition of the model. This is the contribution of each observation to the log likelihood function.
logprob = models.loglogit(V, av, choice)

# Define level of verbosity
logger = msg.bioMessage()
logger.setGeneral()

# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'mnl_2017_baseline_national_exports_20240624'

#biogeme.generateHtml = True
#biogeme.generatePickle = False
results = biogeme.estimate()

# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
pandasResults

[03:04:34] < General >   Remove 17 unused variables from the database as only 19 are used.
[03:04:45] < General >   *** Initial values of the parameters are obtained from the file __mnl_2017_baseline_national_exports_20240624.iter
[03:05:01] < General >   Log likelihood (N = 195603):  -119447.4 Gradient norm:      2e+05 Hessian norm:       1e+12 
[03:05:23] < General >   Log likelihood (N = 195603):  -119431.2 Gradient norm:      6e+03 Hessian norm:       1e+12 
[03:05:47] < General >   Log likelihood (N = 195603):  -119431.2 Gradient norm:          7 Hessian norm:       1e+12 
[03:06:11] < General >   Log likelihood (N = 195603):  -119431.2 Gradient norm:        0.7 Hessian norm:       1e+12 
[03:06:30] < General >   Log likelihood (N = 195603):  -119431.2 Gradient norm:        0.7 Hessian norm:       1e+12 BHHH norm:       1e+13
[03:06:32] < General >   Results saved in file mnl_2017_baseline_national_exports_20240624~01.html
[03:06:32] < General >   Results saved in file mnl_2017_ba

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_AIR,0.878059,0.026020,33.745467,0.000000e+00,0.026648,32.950811,0.000000e+00
ASC_PARCEL,1.347569,0.019987,67.421123,0.000000e+00,0.022014,61.214968,0.000000e+00
ASC_RAIL,-8.427599,0.204200,-41.271232,0.000000e+00,0.194161,-43.405165,0.000000e+00
B_AIR_CNTRY_C,-2.826241,0.022012,-128.397338,0.000000e+00,0.021792,-129.694060,0.000000e+00
B_AIR_CNTRY_M,-2.839392,0.031956,-88.854248,0.000000e+00,0.033161,-85.623525,0.000000e+00
B_AIR_CNTRY_S,-0.590764,0.027890,-21.182161,0.000000e+00,0.027467,-21.508067,0.000000e+00
B_AIR_DIST,0.000532,0.000011,47.506594,0.000000e+00,0.000012,45.509451,0.000000e+00
B_AIR_MG,0.376723,0.022592,16.674698,0.000000e+00,0.022313,16.883830,0.000000e+00
B_AIR_VALDEN,0.000055,0.000007,7.967986,1.554312e-15,0.000018,3.026333,2.475394e-03
B_AIR_WGHT_2,-0.797400,0.019307,-41.301871,0.000000e+00,0.019613,-40.656979,0.000000e+00
